# Mapping with Python

In today's data-driven world, visualizing information geographically can provide valuable insights. In this demonstration, we'll explore how to create insightful maps using Python. By the end of this demonstration, you'll have a visual representation of the employment landscape.

## Dataset Overview:

We'll be working with county-level employment data, focusing on two main metrics: (1) Unemployment Rates and (2) jobs lost due to establishment closures per 1000 inhabitants. We use the average of these variables during the period from 2002 to 2019.

## Importing libraries:
    
Before diving into the code, our first step is to import the necessary libraries. Ensure you've installed them beforehand.

In [ ]:
from folium.features import GeoJsonTooltip
from folium.plugins import HeatMap
from folium import plugins
import geopandas as gpd
import pandas as pd
import numpy as np
#import geoplotlib
import geojson
import chardet
import folium
import branca
import json
import glob

## Opening files:
    
In this section, we'll begin by loading our data files. We'll be utilizing both a GeoJSON and a shapefile specific to the US, alongside our average employment data.

It's worth noting that while we could skip detecting the file encoding for US data, this step could be helpful when working with international datasets. Different countries may use unique encodings and characters, and ensuring compatibility can prevent potential issues down the line.

In [ ]:
shapeFile = "C:/GitHub/cb_2018_us_county_500k.shp"
intensityFile = "C:/GitHub/employment_map.csv"
geojsonFile = 'C:/GitHub/gz_2010_us_050_00_500k.json'

In [ ]:
encSF=chardet.detect(open(shapeFile,'rb').read())['encoding']
encIF=chardet.detect(open(intensityFile,'rb').read())['encoding']
encGF=chardet.detect(open(geojsonFile,'rb').read())['encoding']

In [ ]:
shapeData = gpd.read_file(shapeFile)
intensityData =pd.read_csv(intensityFile)
geojsonData = (open(geojsonFile, "r", encoding=encGF)).read()

## Testing the shapefile:

Now we visually inspect our shapefile. A simple plot will allow us to ensure that the shapefile correctly represents all the counties within the Continental US.

In [ ]:
continental_shapeData = shapeData.cx[-125:-65,25:50]
cUS_SD_plot = continental_shapeData.plot();
cUS_SD_plot.set_frame_on(False);
cUS_SD_plot.set_yticklabels([]);
cUS_SD_plot.set_xticklabels([]);
cUS_SD_plot.set_xticks([]);
cUS_SD_plot.set_yticks([]);


## Testing the geojson file:

Now we visually inspect our geojson. A simple plot will allow us to ensure that the geojson correctly represents all the counties within the US.

In [ ]:
testMap = folium.Map(location=[39,-99],tiles="OpenStreetMap",zoom_start=4)
style_function = lambda x: {'fillColor': '#90EE90',
                            'color': '#006400',
                            'weight': 0.25,
                            'opacity': 0.5,
                            'fillOpacity': 0.2}
folium.GeoJson(data=geojsonData,style_function=style_function).add_to(testMap)
folium.LayerControl().add_to(testMap)
testMap

## Mapping employment

Now that we know the files we have, we can use them to create a map of county-level average employment. First, we open the geojson file again and format the ID so it matches the FIPS code from the employment data. Similarly, we format the FIPS code for the employment data and convert it to string. Then, we merge the two data sets and plot the maps.

In [ ]:
geoJson = json.load(open(geojsonFile, "r", encoding=encGF))
for i in range(len(geoJson['features'])):
    geoJson['features'][i]['properties']['statecountyfp'] = geoJson['features'][i]['properties']['GEO_ID'][9:14]
    if not geoJson['features'][i]['properties'].get('NAME'):
        geoJson['features'][i]['properties']['NAME'] = 'Missing'
gdf = gpd.GeoDataFrame.from_features(geoJson['features'])

In [ ]:
employment = pd.read_csv('C:/GitHub/employment_map.csv')
employment['statecountyfp'] = employment['statecountyfp'].apply(lambda x: "{:05}".format(x))

In [ ]:
df = gdf.merge(employment, left_on='statecountyfp', right_on='statecountyfp', how="outer") 
df = df[~df['geometry'].isna() & ~df['unemp'].isna()]

In [ ]:
choroplethMapU = folium.Map(location=[39,-99], zoom_start=4,tiles='openstreetmap')
custom_scale = (df['unemp'].quantile((0,0.2,0.4,0.6,0.8,0.9,0.95,1))).tolist()
folium.Choropleth(
            geo_data=geojsonData,
            encoding=encGF,
            data=df,
            columns=['GEO_ID','unemp'],
            key_on='feature.properties.GEO_ID',
            threshold_scale=custom_scale,
            fill_color='YlOrRd',
            nan_fill_color="White",
            fill_opacity=0.7,
            line_opacity=0.2,
            legend_name='Average unemployment rate (2003-2019)',
            highlight=True,
            line_color='black').add_to(choroplethMapU)
folium.GeoJson(
    geojsonData,
    style_function=lambda x: {'color': 'transparent', 'fillColor': 'transparent', 'weight': 0},
    tooltip=folium.GeoJsonTooltip(
        fields=['NAME'],
        aliases=[''],
    )
).add_to(choroplethMapU)
choroplethMapU

In [ ]:
choroplethMapD = folium.Map(location=[39,-99], zoom_start=4,tiles='openstreetmap')
custom_scale = (df['dest'].quantile((0,0.05,0.1,0.2,0.4,0.6,0.8,0.9,0.95,1))).tolist()
folium.Choropleth(
            geo_data=geojsonData,
            encoding=encGF,
            data=df,
            columns=['GEO_ID','dest'],
            key_on='feature.properties.GEO_ID',
            threshold_scale=custom_scale,
            fill_color='RdPu',
            nan_fill_color="White",
            fill_opacity=0.7,
            line_opacity=0.2,
            legend_name='Average job destruction due to establishment deaths per 1000 inhabitants (2003-2019)',
            highlight=True,
            line_color='black').add_to(choroplethMapD)
folium.GeoJson(
    geojsonData,
    style_function=lambda x: {'color': 'transparent', 'fillColor': 'transparent', 'weight': 0},
    tooltip=folium.GeoJsonTooltip(
        fields=['NAME'],
        aliases=[''],
    )
).add_to(choroplethMapD)
choroplethMapD

## Exporting to HTML

Now that we have the maps, we can export them to HTML.

In [ ]:
choroplethMapU.save("C:/GitHub/choroplethMapU.html")
choroplethMapD.save("C:/GitHub/choroplethMapD.html")